# 代码表达思想
表明返回类型 ， 表明不修改对象；
使用std算法
如
auto it = std::find(begin(v),end(v),val);

而不是 for
int index = -1；
佛如（interestingi；i《v.size();++i）{
    if（vi[i] == val){
      index =i;
      break;  
    }
}

# 使用ISO 标注你的C++ 代码
文档需要描述实际的行为
不要出现未定义行为，
当必须使用未ISO化扩展时，可用稳定的接口封装

# 表达意图
for (const auto& v:vec){...}  //不修改容器元素，

for (auto& v:vec){...} //可修改元素

std::for_each(std::execution::par, vec, [](auto v) {...});          //并行*std::execution::par*执行，无顺序要求

文档应说明代码做了什么，不是怎么做。



program应当是静态类型安全的

编译器知道数据的类型，可以检测到类型错误

对于 union cast 数组退化、范围错误、窄化转换，有相应办法

c++17后 std::variant 可替代union

基于template的代码减少cast的需要；

当使用C array调用函数时，就会发生数组退化。
函数需要使用指向数组第一个元素的指针，另加数组的长度。
这意味着，从一个类型丰富的C array开始，，以类型差的数组首项指针结束。
c++20开始解决：
std::span
std::span 可以自动推算出CArray大小，也可以防止范围错误的发生

窄化是对算数值的有精度损失的隐式转换。
int i1(2.14);
int i2 = 3.14;

如果使用{}初始化语法，编译器可以检测到*窄化转换*；
int i1{3.14};
int i2 = {3.14};



# 有限选择编译时期检查

C++11开始支持：

static_assert

编译器可以对
static_assert(size(int)>=4) 表达式进行求值，并可产生编译错误。
类型特征（type traits） 库允许程序员表达丰富的条件，
例如
static_assert(std::is_integral<T>::value) 

当static_assert中的表达式求值为faluse 时，编译器会输出一个可读的错误信息



## 其他事项应在运行期检查

dynamic_cast 可以安全的将类的*指针*和*引用*沿着继承层次结构进行：向上、向下、侧向转换。
如果转型失败,
对于指针，得到nullptr;
对于引用，得到std::bad_cast 异常

## 今早识别运行期错误

管理好指针和CArray，检查范围。
检查cast
尽量避免转换，
避免窄化的*转换*及*输入*

## 不要卸扣任何资源

资源泄露对于长期运行的程序尤其致命。资源可以是*内存*、*文件句柄*、*套接字*。
处理资源的惯用法时RAII。
RAII时Resource Acquisition Is Initialization 的缩写。资源获取即初始化。
本质上意味着你在用户定义类型的*构造函数*中获取资源，在*析构函数*中释放资源。
通过使对象成为一个有作用于的对象，C++ 在运行时会自动招呼道资源的生存期。

C++ 大量使用RAII： 锁负责处理互斥量，
智能指针负责处理原始内存，
STL的容器负责处理底层元素。


## 不要浪费时间和空间

如下函数：
void lower(std::string s) {
    for (unsigned int i = 0;i<=std::strlen(s.data());++i){
        s[i] =std::tolower(s[i]);
    }
}

使用stl算法 std::transform
std::transform(s.begin(),s.end(),s.begin(),
    [](char c){return std::tolower(c);  });
// 算法自动确定了字符串的大小。不需要指定长度。


例子2：
为一个用户定义的主句类型生命*拷贝语义*（**拷贝构造**函数，和**拷贝赋值**运算符），
会*抑制*自动定义的*移动语义*（**移动构造函数**和**移动赋值运算符**）.

最终，编译器永远用不了*廉价*的*移动语义*，而只能使用代价高昂的拷贝语义。

struct S{
    std::string s_;
    S(std::string s): s_(s) {}  //**拷贝构造函数**
    S(const S& rhs): s_(rhs.s_){}  //**拷贝构造函数**
    s& operator =(const S& rhs) {s_ =rhs.s_;return *this;}  //**拷贝赋值**运算符


}
S s1;
S s2 = std::move(s1); // 此处实际行为为*拷贝*，不能从s1.s_移动

## 移动构造函数
定义一个空的*构造函数方法*，该方法采用一个*对类类型的右值引用*作为参数，如以下示例所示：

MemoryBlock(MemoryBlock&& other)
   : _data(nullptr)
   , _length(0)
{
    *移动构造函数中，将源对象中的类数据成员添加到要构造的对象：*
_data = other._data;
_length = other._length;

 *将源对象的数据成员分配给默认值。 这可以防止析构函数多次释放资源（如内存）:*
 other._data = nullptr;
other._length = 0;
}

## 移动赋值运算符
定义一个空的赋值运算符，该运算符采用一个*对类类型的右值引用*作为参数并返回一个对类类型的引用，如以下示例所示：

MemoryBlock& operator=(MemoryBlock&& other)
{
*在移动赋值运算符中，如果尝试将对象赋给自身，则添加不执行运算的条件语句。*
if (this != &other)
{
    在条件语句中，从  *被赋值*   对象中释放所有资源（如内存

    //以下示例 将 *被赋值* 的对象中释放_data成员
    // Free the existing resource.
    delete[] _data;


    *将数据成员从源对象转移到要构造的对象*

    // Copy the data pointer and its length from the
    // source object    .
    _data = other._data;
    _length = other._length;

    // Release the data pointer from the source object so that
    // the destructor does not free the memory multiple times.
    other._data = nullptr;
    other._length = 0;

    *返回对当前对象的引用*
    return *this;
}

}

## 若要防止资源泄漏，请始终释放移动赋值运算符中的资源（如内存、文件句柄和套接字）。

## 若要防止不可恢复的资源损坏，请正确处理移动赋值运算符中的自我赋值。

如果为你的类
*同时提供了移动构造函数和移动赋值运算符*
，则可以编写*移动构造函数*来调用**移动赋值运算符**，从而消除冗余代码。 以下示例显示了调用移动赋值运算符的移动构造函数的修改后的版本：

// Move constructor.
MemoryBlock(MemoryBlock&& other) noexcept
   : _data(nullptr)
   , _length(0)
{
   *this = std::move(other); //__调用移动赋值运算符__
}

//*std::move 函数将左值 other 转换为右值。*
*括号内为左值，直接写为右值*
*this = other   //类型不匹配
*this = &other //不行吗？？为何显示调用std::move// 不能正确调用移动赋值运算符吗？


## const 优先
使用常量时，代码更容易验证。
常量有更高的优化潜力，
常量在*并发程序*中有很大优势。

不可变数据在设计上是没有*数据竞争*的。

## 封装杂乱的构建
混乱的代码往往是低级代码，易于隐藏错误，出问题。

尽量使用STL中的高级构建（容器、算法）

把杂乱的代码封装到一个自定义类型、函数中。

## 使用辅助工具
使用静态分析工具、并发工具、测试工具自动完成*验证步骤*。

## 使用设计良好、文档齐全、支持良好的库
c++STL、 、Guidelines支持库
Boost库

